## SimRankLowrank demo

In [73]:
%matplotlib inline
import numpy as np
import scipy.io as sio
import scipy.sparse as spsp
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from pandas import read_csv

In [113]:
graph_filename_mat = "./data/delaunay_n12.mat"
c = 0.3
num_iter = 20
W = sio.loadmat(graph_filename_mat)['Problem'][0][0][2]
C = normalize(W, norm='l1', axis=0)
C = np.sqrt(c) * C
print "Number of vertices =", C.shape[0]
print "Number of edges =", C.nnz / 2

Number of vertices = 4096
Number of edges = 12264


## Compute correct SimRank with naive aproach

In [20]:
def simrank_naive(W, k):
    S = np.eye(max(W.shape))
    for i in range(int(k)):
        S = W.T.dot(S)
        S = W.T.dot(S.T)
        for j in xrange(max(W.shape)):
            S[j,j] = 1
    return S

In [29]:
S_true = simrank_naive(C, num_iter)

## Compute SimRank lowrank approximation

Our approach in SimRank approximation is presented in the following equation:
$$
\tilde{S} = I + C^{\top}C - \text{diag}(C^{\top}C) + UDU^{\top}.
$$
The matrix $C$ is the normalized by columns and scaled by $\sqrt{c}$ adjacency matrix of the graph. The matrix $U$ is lowrank and the matrix $D$ is diagonal.

In [108]:
num_iter = 50
rank = 100
oversampl = 0
U_file = "./data/U_del12.csv"
d_file = "./data/d_del12.csv"
graph_filename_mtx = graph_filename_mat.split("/")[2].split(".")[0] + ".mtx" 
graph_filename_mtx = "/".join(graph_filename_mat.split("/")[0:2]) + "/" + graph_filename_mtx

In [ ]:
!./ccode/build/simrank_lowrank -g $graph_filename_mtx -i $num_iter -c $c -r $rank -s $oversampl -u $U_file -d $d_file

In [110]:
U = read_csv("./data/U_del12.csv", header=None)
d = np.array(read_csv("./data/d_del12.csv", header=None))
D = np.diag(d[:, 0])

In [111]:
CTC = C.T.dot(C)
S_appr = np.eye(C.shape[0]) + CTC - np.diag(CTC.diagonal()) + U.dot(D).dot(U.T)

In [112]:
print "Difference in Frobenius norm =", np.linalg.norm(S_appr - S_true) / np.linalg.norm(S_true)
print "Difference in l2 norm =", np.linalg.norm(S_appr - S_true, 2) / np.linalg.norm(S_true, 2)

Difference in Frobenius norm = 0.0138945330381
Difference in l2 norm = 0.0697796429933
